# Local Interpretable Model-Agnostic Explanations (LIME)

## Library Imports

In [ ]:
import os
import pickle
import warnings

import pandas as pd
from sklearn.preprocessing import LabelEncoder
import lime.lime_tabular
import lime.submodular_pick
import numpy as np

In [ ]:
RAND_STATE = 0

## Importing the Train and Test Sets
We load the both the unencoded and encoded `X` train and test sets.

The unencoded versions are required because we will need to pass them to a `lime` explainer later and, unfortunately, `lime` performs its own scaling and does not handle one-hot encoded features. Hence, they need to be re-encoded differently.

In [ ]:
datasets_folder = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/datasets"

In [ ]:
X_train_unencoded = pd.read_csv(os.path.join(datasets_folder, "obesity_X_train_unencoded.csv"), index_col=0)
X_train_unencoded

In [ ]:
X_test_unencoded = pd.read_csv(os.path.join(datasets_folder, "obesity_X_test_unencoded.csv"), index_col=0)
X_test_unencoded

In [ ]:
X_train = pd.read_csv(os.path.join(datasets_folder, "obesity_X_train.csv"), index_col=0)
X_train

In [ ]:
X_test = pd.read_csv(os.path.join(datasets_folder, "obesity_X_test.csv"), index_col=0)
X_test

We also import `y_test` so that we can choose an instance from each obesity class to explain:

In [ ]:
y_test = pd.read_csv(os.path.join(datasets_folder, "obesity_y_test.csv"), index_col=0)["Obese"]
y_test

## Importing Encoders

We import some of the encoders we will need:

In [ ]:
def import_encoder(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/encoders/{filename}"
    with open(file_path, 'rb') as file: 
        encoder = pickle.load(file)
    print(f"Encoder imported from {file_path}")
    return encoder

In [ ]:
scaler = import_encoder("scaler.pkl")

In [ ]:
nominal_ohe = import_encoder("nominal_ohe.pkl")

In [ ]:
target_le = import_encoder("target_le.pkl")

## Importing the Random Forest Classifiers
We import the random forest classifiers that we trained previously. LIME will be applied to these models to explain their predictions.

In [ ]:
def import_model(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/models/{filename}"
    with open(file_path, 'rb') as file: 
        model = pickle.load(file)
    print(f"Model imported from {file_path}")
    return model

In [ ]:
rand_forest = import_model("rand_forest.pkl")

In [ ]:
rand_forest_no_hw = import_model("rand_forest_no_hw.pkl")

## Preprocessing

`lime` is a little troublesome, unfortunately. When creating an explainer, the train set must be passed to the constructor. However, we must be careful with the encoding of the train set.

First, all categorical features should be encoded using label encoding. One-hot encoding should not be used — instead, one-hot encoding should only be applied through the `predict_fn` argument of `explain_instance()`.

`lime` also does its own feature scaling. We could pre-scale the data using standardisation like we did before. However, `lime` will apply its own feature scaling on top of that. It also becomes difficult to interpret the explainer's results since they will be with respect to the scaled values.

Hence, we will need to preprocess the data into a format suitable for consumption by the explainer. This involves encoding all categorical features using label encoding, but _not_ performing feature scaling. Then, we will have to reprocess the data the explainer passes to `predict_fn` into a format that our trained model understands.

First, we make copies of the unencoded sets:

In [ ]:
X_train_ex = X_train_unencoded.copy()

In [ ]:
X_test_ex = X_test_unencoded.copy()

We encode all categorical features (which all happen to be nominal) using label encoding:

In [ ]:
nominal_features = [
    "Gender",
    "family_history_with_overweight",
    "FAVC",
    "CAEC",
    "SMOKE",
    "SCC",
    "CALC",
    "MTRANS"
]

label_encoders = {feature: None for feature in nominal_features}
for feature in label_encoders.keys():
    le = LabelEncoder()
    feature_all = pd.concat((X_train_unencoded[feature], X_test_unencoded[feature]))
    le.fit(feature_all)
    X_train_ex[feature] = le.transform(X_train_ex[feature])
    X_test_ex[feature] = le.transform(X_test_ex[feature])
    label_encoders[feature] = le

In [ ]:
X_train_ex

In [ ]:
X_test_ex

### Dropping the `Height` and `Weight` Columns
As with the cross-validation and training stages, we create a variant of the `X` sets without the height and weight.

In [ ]:
X_train_ex_no_hw = X_train_ex.drop(["Height", "Weight"], axis=1)
X_train_ex_no_hw

In [ ]:
X_test_ex_no_hw = X_test_ex.drop(["Height", "Weight"], axis=1)
X_test_ex_no_hw

## Converting Between Encodings
We need a function to transform the format of the data back into a format that our models understand. For example, our model trained with the weight column understands data in the following format:

In [ ]:
X_test.iloc[[0]]

But the data the explainer sees is in the following format:

In [ ]:
X_test_ex.iloc[[0]]

More specifically, it sees the data as an `ndarray`:

In [ ]:
X_test_ex.iloc[[0]].to_numpy()

We define a function to convert the explainer's format to our model's format:

In [ ]:
categorical_feature_cols = [X_test_ex.columns.get_loc(feature) for feature in label_encoders.keys()]
numerical_features = ["Age", "Height", "Weight", "FCVC", "NCP", "CH2O", "FAF", "TUE"]
numerical_feature_cols = [X_test_ex.columns.get_loc(feature) for feature in numerical_features]

height_col = X_test_ex.columns.get_loc("Height")
weight_col = X_test_ex.columns.get_loc("Weight")

def transform_for_prediction(ndarr: np.ndarray, with_hw: bool = True):
    copy = ndarr.astype(object)
    
    if not with_hw:
        # A little hacky — the scaler expects the height and weight columns to be there, but they aren't,
        # so we add fake height and weight columns temporarily, then remove them after scaling.
        fake_col_1 = min(height_col, weight_col)
        fake_col_2 = max(height_col, weight_col)
        
        copy_with_fake = np.zeros((copy.shape[0], copy.shape[1] + 2))
        copy_with_fake[:, :fake_col_1] = copy[:, :fake_col_1]
        copy_with_fake[:, fake_col_1 + 1:fake_col_2] = copy[:, fake_col_1:fake_col_2]
        copy_with_fake[:, fake_col_2 + 1:] = copy[:, fake_col_2 - 1:]
        
        copy = copy_with_fake.astype(object)
    
    # Scale numerical features
    copy[:, numerical_feature_cols] = scaler.transform(copy[:, numerical_feature_cols])

    # Undo the label encoding. Remember that all our categorical features happen to be nominal,
    # so we used one-hot encoding instead.
    for (feature, label_encoder), col in zip(label_encoders.items(), categorical_feature_cols):
        copy[:, col] = label_encoder.inverse_transform(copy[:, col].astype("int"))

    # One-hot encode the nominal features.
    oh_encoded = nominal_ohe.transform(copy[:, categorical_feature_cols])

    if with_hw:
        # Drop old columns that have been one-hot encoded.
        copy = np.delete(copy, categorical_feature_cols, axis=1)
    else:
        # Drop old columns that have been one-hot encoded and the fake columns.
        copy = np.delete(copy, categorical_feature_cols + [height_col, weight_col], axis=1)

    # Concatenate with the new one-hot encoded columns.
    copy = np.concatenate((copy, oh_encoded), axis=1)

    return copy.astype("float")

Just to make sure the function works, we perform a sanity check. The two arrays below should be exactly the same:

In [ ]:
X_test.iloc[[0]].to_numpy()

In [ ]:
transform_for_prediction(X_test_ex.iloc[[0]].to_numpy())

And so should the following two:

In [ ]:
X_test.drop(["Height", "Weight"], axis=1).iloc[[0]].to_numpy()

In [ ]:
transform_for_prediction(X_test_ex_no_hw.iloc[[0]].to_numpy(), with_hw=False)

## Creating the Explainers
The data is now in a suitable format. We now create two explainers: one for the `X` data with the weight column, and one for the `X` data without the weight column.

In [ ]:
categorical_names = {X_test_ex.columns.get_loc(feature): list(le.classes_) for feature, le in label_encoders.items()}

In [ ]:
# Convert "No" to "Non-obese" and "Yes" to "Obese" just for better clarity in the generated diagrams.
class_names = ["Non-obese" if cls == "No" else "Obese" for cls in target_le.classes_]

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_ex.to_numpy(),
    feature_names=X_train_ex.columns,
    class_names=class_names,
    categorical_features=categorical_feature_cols,
    categorical_names=categorical_names,
    mode='classification',
    random_state=RAND_STATE
)

In [ ]:
categorical_feature_cols_no_hw = [X_test_ex.drop(["Height", "Weight"], axis=1).columns.get_loc(feature) for feature in label_encoders.keys()]
categorical_names_no_hw = {X_test_ex.drop(["Height", "Weight"], axis=1).columns.get_loc(feature): list(le.classes_) for feature, le in label_encoders.items()}

In [ ]:
explainer_no_hw = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_ex_no_hw.to_numpy(),
    feature_names=X_train_ex_no_hw.columns,
    class_names=class_names,
    categorical_features=categorical_feature_cols_no_hw,
    categorical_names=categorical_names_no_hw,
    mode='classification',
    random_state=RAND_STATE
)

## Explaining Instances
We can finally use the explainers to explain some instances. We randomly choose a non-obese instance and an obese instance:

In [ ]:
X_test_ex_with_oblevel = pd.concat((X_test_ex, y_test), axis=1)
instances = X_test_ex_with_oblevel.groupby("Obese").sample(random_state=RAND_STATE).drop("Obese", axis=1)
instances

In [ ]:
def explain_instances(explainer, instances, model, with_hw=True):
    # Convert the instance to an ndarray.
    instances_arr = instances.to_numpy()
    for instance in instances_arr:
        # The code below will generate warnings about how our encoders were fitted to labelled data,
        # but now we're running them on unlabelled data.
        # These warnings are safe to ignore.
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            explanation = explainer.explain_instance(
                data_row=instance,
                predict_fn=lambda instances: model.predict_proba(transform_for_prediction(instances, with_hw=with_hw)),
                top_labels=1,
            )
            explanation.show_in_notebook(show_table=True, show_all=False)

### With Height and Weight

Now, we explain the model's predictions for the instances using the explainer with height and weight first:

In [ ]:
explain_instances(explainer, instances, rand_forest)

### Without Height and Weight
We now explain the instances with the model trained without the height and weight:

In [ ]:
instances_no_hw = instances.drop(["Height", "Weight"], axis=1)
explain_instances(explainer_no_hw, instances_no_hw, rand_forest_no_hw, with_hw=False)

## Explaining a Representative Set of Instances
To gain global insights into our models' predictions, we can use SP-LIME (submodular-pick LIME), which picks out representative samples from our dataset to explain.
We create SP explainers (**the cells below will take some time to run; please be patient**):

In [ ]:
def create_sp_explainer(explainer, model, X, with_hw=True):
    # The code below will generate tons of warnings about how our encoders were fitted to labelled data, but now we're running them on unlabelled data.
    # These warnings are safe to ignore.
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        return lime.submodular_pick.SubmodularPick(
            explainer,
            X.to_numpy(),
            predict_fn=lambda instances: model.predict_proba(transform_for_prediction(instances, with_hw=with_hw)),
            sample_size=200,
            num_exps_desired=4,
            num_features=len(X.columns)
        )

In [ ]:
sp_explainer = create_sp_explainer(explainer, rand_forest, X_test_ex)

In [ ]:
sp_explainer_no_hw = create_sp_explainer(explainer_no_hw, rand_forest_no_hw, X_test_ex_no_hw, with_hw=False)

### With Weight
We can now show the explanations for these representative samples:

In [ ]:
for exp in sp_explainer.sp_explanations:
    exp.as_pyplot_figure(label=exp.available_labels()[0])

### Without Weight

In [ ]:
for exp in sp_explainer_no_hw.sp_explanations:
    exp.as_pyplot_figure(label=exp.available_labels()[0])